In [1]:
import torch
a=torch.linspace(-50,50,15)
print(a)
torch.sigmoid(a)
#存在梯度离散情况,就是在无穷的时候梯度几乎不变也就很长时间得不到更新

tensor([-5.0000e+01, -4.2857e+01, -3.5714e+01, -2.8571e+01, -2.1429e+01,
        -1.4286e+01, -7.1429e+00,  4.7684e-07,  7.1429e+00,  1.4286e+01,
         2.1429e+01,  2.8571e+01,  3.5714e+01,  4.2857e+01,  5.0000e+01])


tensor([1.9287e-22, 2.4399e-19, 3.0866e-16, 3.9047e-13, 4.9396e-10, 6.2487e-07,
        7.8987e-04, 5.0000e-01, 9.9921e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00])

In [40]:
import torch
from torch import autograd
print('torch:',torch.__version__)

x=torch.tensor(1.)
a=torch.tensor(1.,requires_grad=True)
b=torch.tensor(2.,requires_grad=True)
c=torch.tensor(3.,requires_grad=True)

y=a**2*x+b*x+c

print('before:',a.grad,b.grad,c.grad)
grads=autograd.grad(y,[a,b,c])
print('after:',grads[0],grads[1],grads[2])

torch: 1.7.1
before: None None None
after: tensor(2.) tensor(1.) tensor(1.)


In [2]:
import torch
a=torch.linspace(-50,50,15)
print(a)
torch.tanh(a)#-1-1之间

tensor([-5.0000e+01, -4.2857e+01, -3.5714e+01, -2.8571e+01, -2.1429e+01,
        -1.4286e+01, -7.1429e+00,  4.7684e-07,  7.1429e+00,  1.4286e+01,
         2.1429e+01,  2.8571e+01,  3.5714e+01,  4.2857e+01,  5.0000e+01])


tensor([-1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00, -1.0000e+00,
        -1.0000e+00, -1.0000e+00,  4.7684e-07,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00])

In [3]:
#ReLu激活函数;>0才线性响应;<0不响应
#不存在梯度离散和爆炸的情况,应该优先使用relu函数
a=torch.linspace(-50,50,15)
print(a)
torch.relu(a)#-1-1之间

tensor([-5.0000e+01, -4.2857e+01, -3.5714e+01, -2.8571e+01, -2.1429e+01,
        -1.4286e+01, -7.1429e+00,  4.7684e-07,  7.1429e+00,  1.4286e+01,
         2.1429e+01,  2.8571e+01,  3.5714e+01,  4.2857e+01,  5.0000e+01])


tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 4.7684e-07, 7.1429e+00, 1.4286e+01, 2.1429e+01, 2.8571e+01,
        3.5714e+01, 4.2857e+01, 5.0000e+01])

In [43]:
#MSE均方差
import torch
from   torch.nn import functional as F
x=torch.tensor([1.])
w=torch.tensor([2.])
w.requires_grad_()#表示w需要求导数,b在函数中没有,所以不必设置
mse=F.mse_loss(w*x,torch.ones(1))
#w*x为真实值,torch.ones(1)为假设的预测y
torch.autograd.grad(mse,[w])
#表示mse对w求导

(tensor([2.]),)

In [41]:
#MSE均方差,梯度方法二
import torch
from   torch.nn import functional as F
x=torch.tensor([1.])
w=torch.tensor([2.])
w.requires_grad_()#表示w需要求导数,b在函数中没有,所以不必设置
mse=F.mse_loss(w*x,torch.ones(1))
#w*x为真实值,torch.ones(1)为假设的预测y
mse.backward()#构建图之后反向传播更新梯度
w.grad

tensor([2.])

In [49]:
#softmax函数,pi对aj求梯度
import torch
from   torch.nn import functional as F
a=torch.rand(3)
a.requires_grad_()#需要对a变量求偏导
p=F.softmax(a,dim=0)#构建图同时求pi,一个三个数据的向量
torch.autograd.grad(p[0],[a],retain_graph=True)
torch.autograd.grad(p[1],[a],retain_graph=True)
torch.autograd.grad(p[1],[a])
#每次求梯度会清除,要求下一个梯度的话,retain_graph=True保持状态

(tensor([-0.0895,  0.2166, -0.1271]),)

In [54]:
#单层感知机求梯度
x=torch.rand(1,10)#10个数据的张量
w=torch.rand(1,10,requires_grad=True)#表示要对w球梯度
#构建网络
y=F.sigmoid(x@w.t())#这里线性函数的b为0
print(y.shape)
mse=F.mse_loss(torch.ones(1,1),y)#假设的真实值,预测之y
print(mse.shape)
#y是一维张量,mse是标量,可见mse_loss将两个一维张量之间的计算结果是标量
mse.backward()
w.grad
#输出loss对wi的各个梯度组成的tensor

torch.Size([1, 1])
torch.Size([])


tensor([[-0.0087, -0.0062, -0.0022, -0.0110, -0.0078, -0.0113, -0.0163, -0.0005,
         -0.0076, -0.0064]])

In [55]:
#多层感知机求梯度
x=torch.rand(1,10)#10个数据的张量
w=torch.rand(10,10,requires_grad=True)#表示要对w球梯度,10表示有10个输出,10*10个连线
#构建网络
y=F.sigmoid(x@w.t())#这里线性函数的b为0
print(y.shape)
mse=F.mse_loss(torch.ones(1,1),y)#假设的真实值,预测之y,torch.ones(1,10)庚合适,因为broadcast规则,torch.ones(1,1)也可以
print(mse.shape)
#y是一维张量,mse是标量,可见mse_loss将两个一维张量之间的计算结果是标量
mse.backward()
w.grad
#输出10个loss分别对wi的各个梯度组成的tensor

torch.Size([1, 10])
torch.Size([])


/tmp/ipykernel_2752/2087981861.py:7: UserWarning: Using a target size (torch.Size([1, 10])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mse=F.mse_loss(torch.ones(1,1),y)#假设的真实值,预测之y,torch.ones(1,10)庚合适,因为broadcast规则,torch.ones(1,1)也可以


tensor([[-1.2365e-03, -7.4269e-04, -4.6281e-04, -1.2787e-03, -1.2137e-03,
         -4.1478e-04, -7.8171e-04, -3.1553e-04, -1.3919e-03, -3.9727e-05],
        [-8.9075e-04, -5.3502e-04, -3.3340e-04, -9.2117e-04, -8.7435e-04,
         -2.9880e-04, -5.6313e-04, -2.2730e-04, -1.0027e-03, -2.8619e-05],
        [-2.8155e-04, -1.6911e-04, -1.0538e-04, -2.9116e-04, -2.7637e-04,
         -9.4444e-05, -1.7799e-04, -7.1845e-05, -3.1692e-04, -9.0458e-06],
        [-2.1135e-03, -1.2695e-03, -7.9108e-04, -2.1857e-03, -2.0746e-03,
         -7.0897e-04, -1.3362e-03, -5.3932e-04, -2.3791e-03, -6.7905e-05],
        [-3.9488e-04, -2.3718e-04, -1.4780e-04, -4.0836e-04, -3.8761e-04,
         -1.3246e-04, -2.4964e-04, -1.0076e-04, -4.4449e-04, -1.2687e-05],
        [-1.2566e-04, -7.5474e-05, -4.7033e-05, -1.2995e-04, -1.2334e-04,
         -4.2151e-05, -7.9440e-05, -3.2065e-05, -1.4144e-04, -4.0372e-06],
        [-7.3081e-04, -4.3895e-04, -2.7354e-04, -7.5576e-04, -7.1735e-04,
         -2.4514e-04, -4.6201e-0